### Test run

In [ ]:
# Import neccsessary luibraries and first data

import numpy as np
import matplotlib.pyplot as plt

f = plt.figure()
f.set_dpi(150)

import geojson
with open('./pahma.geojson') as f:
    parsed_geojson = geojson.load(f)
    
selected_feature = 2
features = parsed_geojson['features'][selected_feature]
print(f"Count of features: {len(parsed_geojson['features'])}")
print(f"Selected feature: {selected_feature}")
geometry = features['geometry']
coordinates = geometry['coordinates']
print(f"Count of points: {len(coordinates)}")

river_X = list(zip(*coordinates))[0]
river_Y = list(zip(*coordinates))[1] 

plt.axis('equal')

plt.plot(river_X, river_Y, 'black', linewidth = 1)
plt.show()